# Generate Embeddings

🎯 Clean text data, generate embeddings and store them in Chroma DB

In [1]:
import sys
sys.executable

'/Users/panos/Library/Caches/pypoetry/virtualenvs/omnibot-lHmdBll9-py3.9/bin/python'

In [2]:
import os 
import re
from tqdm import tqdm

## Clean files

In [ ]:
# Clean Data
def clean_webpage_data(file_path):
    
    # Common words to drop
    logo = ["A.C. LASKARIDIS CHARITABLE FOUNDATION", "ΚΟΙΝΩΦΕΛΕΣ ΙΔΡΥΜΑ ΑΘΑΝΑΣΙΟΣ Κ. ΛΑΣΚΑΡΙΔΗΣ"]
    
    header = ["THE FOUNDATION", "ΤΟ ΙΔΡΥΜΑ",
              "SEA", "ΘΑΛΑΣΣΑ",
              "ENTREPRENEURSHIP", "ΕΠΙΧΕΙΡΗΜΑΤΙΚΟΤΗΤΑ",
              "EDUCATION", "ΕΚΠΑΙΔΕΥΣΗ",
              "EVENTS", "ΕΚΔΗΛΩΣΕΙΣ",
              "COLLABORATIONS", "ΣΥΝΕΡΓΑΣΙΕΣ",
              "OTHER PROJECTS", "ΑΛΛΑ ΠΡΟΓΡΑΜΜΑΤΑ"]
    
    menu = ["OUR TEAM", "Η ΟΜΑΔΑ ΜΑΣ",
            "SUPPORTERS", "ΥΠΟΣΤΗΡΙΚΤΕΣ",
            "NEWS", "ΝΕΑ", 
            "LATEST NEWS", "ΤΕΛΕΥΤΑΙΑ ΝΕΑ"]
    
    footer = ["CONTACT", "ΕΠΙΚΟΙΝΩΝΙΑ",
              "EVENTS", "ΕΚΔΗΛΩΣΕΙΣ",
              "AWARDS", "ΒΡΑΒΕΙΑ", "2020 aclcfWebsite by Cloudevo"]
    
    for file_name in tqdm(os.listdir(file_path)):
        if file_name.endswith(".txt"):
            file = os.path.join(file_path, file_name)
            with open(file, "r") as f:
                text = f.read()
            
            # Drop words
            text = drop_words(text, logo)
            text = drop_words(text, header)
            text = drop_words(text, menu)
            text = drop_words(text, footer)
            
            # Remove extra whitespace and newlines
            text = re.sub(r'\s+', ' ', text).strip()
            
            # Drop Everything before OTHER PROJECTS
            text = drop_before(text, "OTHER PROJECTS")
            
            # Drop Everything after latest news
            text = drop_after(text, "LATEST NEWS")
            
            with open(file, "w") as f:
                f.write(text)
    
    return None

In [ ]:
def drop_words(text, words):
    for word in words:
        text = text.replace(word, "")

    return text

In [ ]:
def drop_before(text, keyword):
    index = text.find(keyword)
    if index != -1:
        text = text[index:]
    
    return text

In [ ]:
def drop_after(text, keyword):
    index = text.find(keyword)
    if index != -1:
        text = text[:index]
    
    return text

In [ ]:
# Provide the path to the folder containing the text files
folder_path = "../data/"
clean_webpage_data(folder_path)

## Generate Embeddings

In [3]:
# Load all txt files from a directory
from langchain.document_loaders import DirectoryLoader, TextLoader
loader = DirectoryLoader('../data/', glob = "./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [4]:
documents

[Document(page_content='https://www.aclcf.org/the-team/chloi-maria-laskaridi/ | Χλόη Μαρία Λασκαρίδη Μέλος Δ.Σ. Η Χλόη κατέχει πτυχίο (Β.Α.) στις Διεθνείς Σχέσεις και στην Ιστορία από το Πανεπιστήμιο Reading της Μεγάλης Βρετανίας. Κατέχει, επίσης, δύο μεταπτυχιακούς τίτλους σε War Studies (ΜΑ) από το King’s College στο Λονδίνο και στη Διαχείριση Ανθρώπινου Δυναμικού (MSc) από το London School of Economics. Από το 2008 εκτελεί καθήκοντα Διευθύντριας Εταιρικής Ανάπτυξης στην εταιρία Ελληνικά Ξενοδοχεία Λάμψα Α.Ε. Είναι μέλος Δ.Σ. του οργανισμού The People’s Trust. Οι Συνεργάτες Μας ΤΕΛΕΥΤΑΙΑ ΠΑΡΟΥΣΙΑΣΗ ΣΥΛΛΟΓΙΚΗΣ ΕΚΔΟΣΗΣ ΠΑΝΔΗΜΙΑ, ΚΑΙ ΑΝΘΡΩΠΟΣ ΣΤΗΝ ΕΛΛΑΔΑ ΕΝΕΡΓΩ ΩΣ ΠΟΛΙΤΗΣ: ΕΝΑ ΕΚΠΑΙΔΕΥΤΙΚΟ ΠΡΟΓΡΑΜΜΑ ΓΙΑ ΤΗ ΔΙΑΜΟΡΦΩΣΗ ΤΩΝ ΜΑΘΗΤΩΝ/ΤΡΙΩΝ ΩΣ ΕΝΕΡΓΩΝ ΠΟΛΙΤΩΝ ΣΥΝΕΔΡΙΟ: Η ΕΥΕΡΓΕΣΙΑ ΚΑΙ Ο ΝΕΟΤΕΡΟΣ ΕΛΛΗΝΙΣΜΟΣ Ο ΔΕΥΤΕΡΟΣ ΜΑΡΑΘΩΝΙΟΣ ΚΑΙΝΟΤΟΜΙΑΣ, ΑΝΑΠΤΥΞΗΣ ΛΥΣΕΩΝ ΚΑΙ ΕΦΑΡΜΟΓΩΝ ΓΙΑ ΜΙΑ ΒΙΩΣΙΜΗ ΚΑΘΗΜΕΡΙΝΟΤΗΤΑ', metadata={'source': '../data/29.txt'}),
 Document(page_content='https://www.aclcf.org/even

In [8]:
# Splitting the text into 
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=20)

texts = text_splitter.split_documents(documents)
print(f'There are {len(texts)} documents')

There are 621 documents


In [9]:
print(texts[11])
print(texts[112])
print(texts[222])

page_content='στα πλαστικά μίας χρήσης, στο πλαίσιο του προγράμματος SeaChange Greek Islands, και να γίνει το πρώτο Single Use Plastic Free νησί. Χάρη στην άριστη συνεργασία του Ιδρύματος με τους τρεις συλλόγους του νησιού, η Δονούσα δεσμεύτηκε για τη μείωση και σταδιακή κατάργηση των πλαστικών μίας χρήσης από τον Απρίλιο 2019. Σήμερα οι περισσότερες επιχειρήσεις του νησιού συμμετέχουν στο πρόγραμμα, κάνοντας' metadata={'source': '../data/128.txt'}
page_content='https://www.aclcf.org/about-en/the-foundation/ | The Athanasios C. Laskaridis Charitable Foundation was established in 2015 in order to act as a catalyst for important issues concerning Greece and Hellenism. Its action focuses on the protection of the marine environment, the support of employment and micro-entrepreneurship as well as the fostering of initiatives, which defend collective interest' metadata={'source': '../data/62.txt'}
page_content='ΤΗ ΔΙΑΜΟΡΦΩΣΗ ΤΩΝ ΜΑΘΗΤΩΝ/ΤΡΙΩΝ ΩΣ ΕΝΕΡΓΩΝ ΠΟΛΙΤΩΝ ΣΥΝΕΔΡΙΟ: Η ΕΥΕΡΓΕΣΙΑ ΚΑΙ Ο ΝΕ

In [10]:
## LOCAL .env file
from dotenv import dotenv_values
secrets = dotenv_values("../.env")
OPENAI_API_KEY = secrets['OPENAI_API_KEY']

## Step 2: Initialize Embeddings engine
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [11]:
## Step 3: Initialize and Persist ChromDB with embeddings
from langchain.vectorstores import Chroma

# Supplying a persist_directory will store the embeddings on disk
persist_directory = '../db'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()
vectordb = None

Using embedded DuckDB with persistence: data will be stored in: ../db
